In [2]:
'''Imports'''

from __future__ import annotations
import random

'''Classes'''

# Vacuum Environment Class
class VacuumEnvironment:
    def __init__(self, width, height, dirt = None) -> None:
        self.width = width
        self.height = height
        self.locations = []
        self.dirt = []
        self.genDirt()
        self.genLocations()

    #Generates a list of all valid locations within the environment
    def genLocations(self):
        for x in range(self.width):
            for y in range(self.height):
                self.locations.append((x,y))                 

    #Generates a list of dirty locations inside the environment
    def genDirt(self):
        for x in range(self.width):
            for y in range(self.height):
                if random.randint(0,1) == 1:
                    self.dirt.append((x,y))
    
    #Removes a location from the dirt list
    def cleanLocation(self, agent: VacuumAgent, location):
        if location in self.dirt:
            self.dirt.remove(location)

    #Checks the dirt/clean state of a given location
    def isClean(self, location):
        if location in self.dirt:
            return False
        else:
            return True
      
                
# Vacuum Agent Class
class VacuumAgent:
    def __init__(self, program = 'ReflexAgent', defaultLocation = (0, 0)) -> None:
        self.bump = False
        self.program = program
        self.performance = 0
        self.location = defaultLocation
        #State only used for when programming is not reflex or random based
        self.map = [] 

    #The sensors available to the agent are accessed through this function
    def sense(self, env: VacuumEnvironment):
        if env.isClean(self.location):
            return 'Clean'
        else:
            return 'Dirty'

    #All valid moves are sensed with a given step size, NOT in use for this agent    
    def validMove(self, env: VacuumEnvironment, step = 1):
        moveloc = []
        for loc in env.locations:
            x = loc[0]
            y = loc[1]

            if(y == self.location[1]):
                if x == self.location[0] + step:
                    moveloc.append(loc)
                elif x == self.location[0] - step:
                    moveloc.append(loc)
                    
            elif(x == self.location[0]):
                if y == self.location[1] + step:
                    moveloc.append(loc)
                elif y == self.location[1] - step:
                    moveloc.append(loc)

        return moveloc
    
    #The actuators available to the agent are accessed through this function
    def actuate(self, action, env: VacuumEnvironment,):
        prevLoc = self.location

        if action == 'Right':
            self.location = (self.location[0] + 1, self.location[1])
            self.performance -= 1
        elif action == 'Left':
            self.location = (self.location[0] - 1, self.location[1])
            self.performance -= 1
        elif action == 'Up':
            self.location = (self.location[0], self.location[1] + 1)
            self.performance -= 1
        elif action == 'Down':
            self.location = (self.location[0], self.location[1] - 1)
            self.performance -= 1
        elif action == 'Suck':
            if env.isClean(self.location) == False:
                self.performance += 10
                env.cleanLocation(self, self.location)

        if(self.location not in env.locations):
            self.bump = True
            self.location = prevLoc

    #For the use of the state program of this agent. Adds a coordinate to the state list of invalid coordinates to remember not to use       
    def bumpListAppend(self, action):
        if action == 'Right':
            self.map.append((self.location[0] + 1, self.location[1]))

        elif action == 'Left':
            self.map.append((self.location[0] - 1, self.location[1]))

        elif action == 'Up':
            self.map.append((self.location[0], self.location[1] + 1))

        elif action == 'Down':
            self.map.append((self.location[0], self.location[1] - 1))

    #For the use of the state program of this agent. Checks if a given coordinate is in the bump list.
    def checkBumpList(self, location):
        if(location in self.map):
            return True
        else:
            return False

    #Given an action returns the resulting location of the agent
    def actionToLocation(self, action) -> tuple:
        if action == 'Right':
            return (self.location[0] + 1, self.location[1])

        elif action == 'Left':
            return (self.location[0] - 1, self.location[1])

        elif action == 'Up':
            return (self.location[0], self.location[1] + 1)

        elif action == 'Down':
            return (self.location[0], self.location[1] - 1)

    #Makes the agent take a step in the environment. Currently senses and takes an action.
    def act(self, env: VacuumEnvironment) -> str:
        if self.program == 'Reflex':

            actions = ['Right','Left','Suck']
            
            #Cleaning
            if self.sense(env) == 'Dirty':
                self.actuate(actions[2], env)
                return actions[2]

            #Movement
            elif self.sense(env) != 'Dirty': 
                if self.location in env.locations:
                    self.actuate(actions[0], env)
                    return actions[0]
                
        elif self.program == 'Reflex_2D':

            actions = ['Right','Left','Up','Down','Suck']
            
            #Cleaning
            if self.sense(env) == 'Dirty':
                self.actuate(actions[4], env)
                return actions[4]

            #Movement
            elif self.sense(env) != 'Dirty': 
                if self.location in env.locations:
                    random_move = random.choice(actions[:-1])
                    self.actuate(random_move, env)
                    return random_move
        
            
        elif self.program == 'Random':
            #Sense current location
            #If dirty clean
            if self.sense(env) == 'Dirty':
                self.actuate('Suck', env)
                return 'Suck'

            #Random Action to change location
            else:
                actions = ['Right','Left','Up','Down']
                randomAction = random.choice(actions)
                self.actuate(randomAction, env)
                return randomAction
    
        elif self.program == 'State':
            #Sense current location
            #If dirty clean
            if self.sense(env) == 'Dirty':
                self.actuate('Suck', env)
                return 'Suck'

            #Random actions to change location
            #Check if coordinate is stored in state
            #If not, perform selected action
           #If location did not change add coordinate as invalid to the state

            validChoice  = False

            while(validChoice == False):
                actions = ['Right','Left','Up','Down']
                randomAction = random.choice(actions)
                if(self.checkBumpList(self.actionToLocation(randomAction)) == False):
                    self.actuate(randomAction, env)
                    validChoice = True

            if(self.bump == True):
                self.bumpListAppend(randomAction)
                self.bump =  False
            
            return randomAction 

In [7]:
#Exercise 2.11


WIDTH, HEIGHT = 2, 1

# Environment
env = VacuumEnvironment(WIDTH, HEIGHT)
print("Environment initialized successfully.")
print(f"Initial dirty locations: {env.dirt}\n")

# Limits
MAX_ACTIONS = 50
steps = 0

# Agent
agent = VacuumAgent(program='Reflex')

# Agent actiong in environment
while env.dirt and steps < MAX_ACTIONS:
    steps += 1
    action = agent.act(env)
    print(f"Action actiong in environment {steps}: {action}, Remaining dirt: {env.dirt}")


print(f"Reflex Agent performance after {steps} steps: {agent.performance}")


Environment initialized successfully.
Initial dirty locations: []

Reflex Agent performance after 0 steps: 0


In [10]:
# Exercise 2.14

# Question B
# Randomized Agent

def vacuum_modular_simulator(width, height, agent_type, action_limit):
    environment = VacuumEnvironment(width, height)
    print(f'Created environment with size ({width}x{height})')
    print(f'Initial Dirty Configurations: {environment.dirt}\n')
    
    agent = VacuumAgent(agent_type)
    actions = 0
    
    while environment.dirt and actions < action_limit:
        actions += 1
        agent.act(environment)
    
    print(f'{agent_type} Reflex Agent performance after {actions} actions: {agent.performance}')
    print(f'Remaining dirty Configurations: {environment.dirt}\n')


vacuum_modular_simulator(3, 3, 'Random', 50)
vacuum_modular_simulator(5, 5, 'Random', 125)


Created environment with size (3x3)
Initial Dirty Configurations: [(0, 2), (2, 0), (2, 1)]

Random Reflex Agent performance after 29 actions: 4
Remaining dirty Configurations: []

Created environment with size (5x5)
Initial Dirty Configurations: [(0, 2), (1, 0), (1, 2), (2, 1), (2, 2), (2, 4), (3, 0), (3, 3)]

Random Reflex Agent performance after 125 actions: -48
Remaining dirty Configurations: [(3, 0)]



In [14]:
# Exercise 2.14

# Question B
# Simple Reflex Agent

def vacuum_modular_simulator(width, height, agent_type, action_limit):
    environment = VacuumEnvironment(width, height)
    agent = VacuumAgent(agent_type)
    
    print(f'--- Running {agent_type} Agent in a {width}x{height} environment ---')
    print(f'The environment has been successfully created\n')
    print(f'Dirty Configurations: {environment.dirt}\n')

    actions = 0
    while environment.dirt and actions < action_limit:
        actions += 1
        agent.act(environment)

    print(f'{agent_type} Agent performance after {actions} actions: {agent.performance}')
    print(f'Remaining dirty Configurations: {environment.dirt}\n')

# Run simulations with different environment sizes
vacuum_modular_simulator(3, 3, 'Reflex_2D', 50)
vacuum_modular_simulator(5, 5, 'Reflex_2D', 125)


--- Running Reflex_2D Agent in a 3x3 environment ---
The environment has been successfully created

Dirty Configurations: [(0, 0), (1, 1), (1, 2), (2, 0), (2, 2)]

Reflex_2D Agent performance after 17 actions: 38
Remaining dirty Configurations: []

--- Running Reflex_2D Agent in a 5x5 environment ---
The environment has been successfully created

Dirty Configurations: [(0, 0), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (3, 0), (3, 2), (3, 3), (4, 2), (4, 3)]

Reflex_2D Agent performance after 120 actions: 12
Remaining dirty Configurations: []



In [15]:
# Exercise 2.14

# Question C

# Environment
width, height = 6, 6
environment = VacuumEnvironment(width, height)

print("The environment has been successfully created\n")
print(f"Initial dirty configurations: {environment.dirt}\n")

# Agent
max_actions = 125
steps_taken = 0
vacuum_agent = VacuumAgent(program='Random')

while environment.dirt and steps_taken < max_actions:
    steps_taken += 1
    vacuum_agent.act(environment)

print(f"Random Agent performance after {steps_taken} actions: {vacuum_agent.performance}")
print(f"Remaining dirty configurations: {environment.dirt}")


The environment has been successfully created

Initial dirty configurations: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 1), (1, 3), (2, 0), (2, 1), (2, 4), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 0), (4, 2), (5, 2), (5, 4), (5, 5)]

Random Agent performance after 125 actions: 29
Remaining dirty configurations: [(2, 0), (2, 1), (3, 0), (3, 1), (4, 0), (5, 2)]


In [17]:

# Exercise 2.14

# Question D Test 1

# Environment
width = 10
height = 10
env = VacuumEnvironment(width, height)

print(f'The environment has been successfully created')
print()
print(f'Dirty configurations: {env.dirt}')
print()

# Parameters
limit = 1000
actions = 0
state_reflex_agent = VacuumAgent('State')

while env.dirt and actions < limit:
    actions += 1
    
    state_reflex_agent.act(env)  

print(f'State Reflex Agent performance after {actions} actions is: {state_reflex_agent.performance}')
print(f'Remaining dirty configurations: {env.dirt}') 


The environment has been successfully created

Dirty configurations: [(0, 0), (0, 1), (0, 3), (0, 5), (0, 6), (1, 0), (1, 1), (1, 7), (1, 8), (2, 1), (2, 2), (2, 6), (2, 7), (3, 0), (3, 2), (3, 3), (3, 5), (3, 8), (3, 9), (4, 2), (4, 3), (4, 5), (4, 6), (4, 8), (4, 9), (5, 0), (5, 1), (5, 2), (5, 6), (5, 9), (6, 1), (6, 3), (6, 5), (7, 0), (7, 3), (7, 4), (7, 5), (7, 7), (7, 8), (8, 0), (8, 2), (8, 3), (8, 4), (8, 5), (8, 7), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5)]

State Reflex Agent performance after 895 actions is: -334
Remaining dirty configurations: []


In [18]:

# Exercise 2.14

# Question D Test 2

# Environment
width = 20
height = 20
env = VacuumEnvironment(width, height)

print(f'The environment has been successfully created')
print()
print(f'Dirty configurations: {env.dirt}')
print()

# Parameters
limit = 1000
actions = 0
state_reflex_agent = VacuumAgent('State')

while env.dirt and actions < limit:
    actions += 1
    
    state_reflex_agent.act(env)  

print(f'State Reflex Agent performance after {actions} actions is: {state_reflex_agent.performance}')
print(f'Remaining dirty configurations: {env.dirt}') 


The environment has been successfully created

Dirty configurations: [(0, 0), (0, 1), (0, 4), (0, 5), (0, 8), (0, 11), (0, 12), (0, 14), (0, 18), (1, 5), (1, 8), (1, 12), (2, 0), (2, 1), (2, 3), (2, 4), (2, 5), (2, 8), (2, 11), (2, 12), (2, 13), (2, 14), (2, 16), (2, 18), (2, 19), (3, 0), (3, 2), (3, 3), (3, 4), (3, 7), (3, 9), (3, 13), (3, 14), (3, 15), (4, 4), (4, 6), (4, 7), (4, 12), (4, 13), (4, 15), (4, 16), (4, 18), (4, 19), (5, 0), (5, 2), (5, 3), (5, 4), (5, 6), (5, 8), (5, 9), (5, 10), (5, 13), (5, 19), (6, 0), (6, 3), (6, 6), (6, 8), (6, 11), (6, 12), (6, 15), (6, 17), (6, 18), (7, 3), (7, 7), (7, 8), (7, 10), (7, 11), (7, 15), (7, 16), (7, 18), (7, 19), (8, 1), (8, 4), (8, 6), (8, 9), (8, 10), (8, 12), (8, 13), (8, 15), (8, 17), (8, 19), (9, 1), (9, 3), (9, 4), (9, 8), (9, 14), (9, 16), (9, 17), (9, 19), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 9), (10, 10), (10, 11), (10, 12), (10, 13), (10, 15), (10, 18), (10, 19), (11, 0), (11, 2), (11, 4), (11, 6), (11,

In [19]:

# Exercise 2.14

# Question D Test 3

# Environment
width = 50
height = 50
env = VacuumEnvironment(width, height)

print(f'The environment has been successfully created')
print()
print(f'Dirty configurations: {env.dirt}')
print()

# Parameters
limit = 1000
actions = 0
state_reflex_agent = VacuumAgent('State')

while env.dirt and actions < limit:
    actions += 1
    
    state_reflex_agent.act(env)  

print(f'State Reflex Agent performance after {actions} actions is: {state_reflex_agent.performance}')
print(f'Remaining dirty configurations: {env.dirt}') 


The environment has been successfully created

Dirty configurations: [(0, 0), (0, 1), (0, 5), (0, 7), (0, 8), (0, 9), (0, 11), (0, 13), (0, 14), (0, 15), (0, 17), (0, 21), (0, 22), (0, 25), (0, 27), (0, 28), (0, 29), (0, 30), (0, 32), (0, 33), (0, 34), (0, 36), (0, 40), (0, 44), (0, 45), (0, 46), (0, 47), (0, 49), (1, 0), (1, 2), (1, 7), (1, 9), (1, 10), (1, 12), (1, 13), (1, 14), (1, 15), (1, 17), (1, 18), (1, 20), (1, 21), (1, 22), (1, 24), (1, 25), (1, 27), (1, 29), (1, 31), (1, 35), (1, 36), (1, 39), (1, 41), (1, 42), (1, 45), (1, 47), (1, 49), (2, 0), (2, 3), (2, 4), (2, 6), (2, 7), (2, 9), (2, 12), (2, 13), (2, 14), (2, 17), (2, 20), (2, 22), (2, 23), (2, 25), (2, 29), (2, 33), (2, 35), (2, 39), (2, 42), (2, 48), (3, 0), (3, 3), (3, 5), (3, 6), (3, 8), (3, 12), (3, 15), (3, 16), (3, 17), (3, 19), (3, 22), (3, 25), (3, 26), (3, 29), (3, 30), (3, 34), (3, 37), (3, 38), (3, 40), (3, 41), (3, 43), (3, 44), (3, 46), (3, 48), (3, 49), (4, 0), (4, 8), (4, 9), (4, 10), (4, 12), (4, 14), 